In [1]:
"""
This script is used to extract the 2000 - 2020 BLS QCEW Industry files for NAICS code 492 (Couriers and messengers) for all US counties. 
There is a significant memory cost to pull this data. 

Pulls avg annual employment and total_annual_wages for nacis code 492 for all US counties 
"""
import pandas as pd
import os
import sys
from datetime import datetime
import yaml
import time
import requests
import urllib
import zipfile
import pprint
from tqdm import tqdm
from glob import glob
import urllib.request
#from platforms.connect.snowpy import SnowPy

Key notes about the data: 
If a county does not the estabishment counts are truly zero, they are not reported 

A dash "-" means the macro cell field (data point combining area/ownership/industry) does not exist for that specific quarter and year.

Suppressed data fields are published with an "N" in the disclosure code field. Only establishment counts are disclosed for these cells, based on approval from this Federal Register Notice, while all other data items for the cell are suppressed (zero-filled).


https://data.bls.gov/cew/doc/access/csv_data_slices.htm#ANNUAL_LAYOUT

https://data.bls.gov/cew/doc/access/data_access_examples.htm#PYTHON

https://www.bls.gov/cew/downloadable-data-files.htm

https://www.bls.gov/cew/questions-and-answers.htm -> suppression addressed in questions 13 & 14


Notes:
BLS does not disclose their detailed methodology for suppression because they want to prevent anyone from reverse-engineering the data in order to get to the suppressed numbers. So basically, we don't really know all the little things they are doing to protect confidentiality.

Primary suppression (dubbed the 80/3 rule) occurs when one of the following conditions is true:

1. There are fewer than three establishments in the given industry for a geographic area.
2. One firm constitutes more than 80 percent of area employment in a given industry

For post 2013, the indsutry csv file for NAICS 492 is accesible by a single url

In [2]:
#URL structure for the csv files from 2014 on: http://data.bls.gov/cew/data/api/[YEAR]/1/area/[NAICS CODE].csv
#replace '1' with any quarter you want or 'a' for annual averages 
#example URL: http://data.bls.gov/cew/data/api/2017/1/area/US000.csv
#Source: https://data.bls.gov/cew/doc/access/csv_data_slices.htm#ANNUAL_LAYOUT

df_2020 = pd.read_csv('https://data.bls.gov/cew/data/api/2020/a/industry/492.csv')
df_2019 = pd.read_csv('https://data.bls.gov/cew/data/api/2019/a/industry/492.csv')
df_2018 = pd.read_csv('https://data.bls.gov/cew/data/api/2018/a/industry/492.csv')
df_2017 = pd.read_csv('https://data.bls.gov/cew/data/api/2017/a/industry/492.csv')
df_2016 = pd.read_csv('https://data.bls.gov/cew/data/api/2016/a/industry/492.csv')
df_2015 = pd.read_csv('https://data.bls.gov/cew/data/api/2015/a/industry/492.csv')
df_2014 = pd.read_csv('https://data.bls.gov/cew/data/api/2014/a/industry/492.csv')

In [3]:
df_2015

,area_fips,own_code,industry_code,agglvl_code,size_code,year,qtr,disclosure_code,annual_avg_estabs,annual_avg_emplvl,...,oty_total_annual_wages_chg,oty_total_annual_wages_pct_chg,oty_taxable_annual_wages_chg,oty_taxable_annual_wages_pct_chg,oty_annual_contributions_chg,oty_annual_contributions_pct_chg,oty_annual_avg_wkly_wage_chg,oty_annual_avg_wkly_wage_pct_chg,oty_avg_annual_pay_chg,oty_avg_annual_pay_pct_chg
0,01000,5,492,55,0,2015,A,NaN,247,5784,...,13252009,5.8,3122259,6.6,15362,2.7,5,0.6,295,0.7
1,01003,5,492,75,0,2015,A,NaN,12,173,...,953794,15.2,326723,24.0,4529,20.0,42,5.5,2152,5.4
2,01009,5,492,75,0,2015,A,N,1,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,01015,5,492,75,0,2015,A,N,9,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,01025,5,492,75,0,2015,A,N,1,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,C4966,5,492,45,0,2015,A,NaN,26,550,...,538913,2.7,89118,1.8,6017,15.9,-21,-2.8,-1109,-2.9
2210,C4970,5,492,45,0,2015,A,N,2,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2211,C4974,5,492,45,0,2015,A,NaN,13,180,...,813201,14.0,250088,20.2,4740,38.8,48,7.3,2450,7.1
2212,US000,3,492,15,0,2015,A,N,3,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


For pre 2013 files, we need to download the industry zip file for hte year and extract the csv for NAICS code 493

In [4]:
print(os.getcwd())
datapath = os.getcwd() + '/qcew_zips/'

C:\Users\hrowe\Documents\FHWA mobility trend report\T4 - Forecasting\Year 2\modeling code\etl


In [4]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

In [5]:
# Define function to download URL to a file
def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

In [6]:
def try_download(url, datapath, file):
    #makedir_if_needed(datapath)
    try:
        download_url(url, file)
        print(f"Downloaded {url} to {file}")
    except urllib.error.HTTPError as e:
        print(f"Couldn't find {url}, Exception: {e}")

In [7]:
def uncompress(filepath):
    # Uncompress if zip file
    if filepath[-4:].lower() == '.zip':
        zipfolder = filepath.split('/')[-1].split('.')[0]
        print(f'                Uncompressing zip file to folder {zipfolder}')
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            zip_ref.extractall(datapath + zipfolder)

In [14]:
#states = pd.read_excel('ETL_Pipeline/ACS/State_Abbrev_FIPS.xlsx')
#states.head()

#https://data.bls.gov/cew/data/files/2009/csv/2009_annual_by_industry.zip

urlbase = 'https://data.bls.gov/cew/data/files/'

def download_qcew_files():
    for year in range(2000, 2014, 1):
        print(year)
        file = f'{datapath}{year}_annual_by_industry.zip'
        if os.path.exists(file) == False:
            url = f'{urlbase}{year}/csv/{year}_annual_by_industry.zip'
            try_download(url, datapath, file)
            #url = f'{urlbase}{year}/csv/{Year}_annual_by_industry.zip'
            #try_download(url, datapath, file)
        
        file = f'{datapath}{year}_annual_by_industry.zip' 
        if os.path.exists(file) == False:
            url = f'{urlbase}{year}/csv/{year}_annual_by_industry.zip'
            try_download(url, datapath, file)
            #url = f'{urlbase}{year}/csv_pus.zip'
            #try_download(url, datapath, file)

In [5]:
#download_qcew_files()

In [17]:
def uncompress_qcew_files():
    for year in range(2000,2014,1):
        try:
            uncompress(datapath + str(year) + '_annual_by_industry.zip' )
            os.remove(datapath + str(year) + '_annual_by_industry.zip')
        except:
            print('unable to uncompress')

        #try:
            #uncompress(datapath + 'pums_' + str(year) + '_csv_pus.zip')
            #os.remove(datapath + 'pums_' + str(year) + '_csv_pus.zip')
        #except:
            #print('unable to uncompress')

In [16]:
#uncompress_qcew_files()

                Uncompressing zip file to folder 2000_annual_by_industry
                Uncompressing zip file to folder 2001_annual_by_industry
                Uncompressing zip file to folder 2002_annual_by_industry
                Uncompressing zip file to folder 2003_annual_by_industry
                Uncompressing zip file to folder 2004_annual_by_industry


In [5]:
path_2000 = datapath + '2000_annual_by_industry/2000.annual.by_industry/2000.annual 492 Couriers and messengers.csv'
path_2001 = datapath + '2001_annual_by_industry/2001.annual.by_industry/2001.annual 492 Couriers and messengers.csv'
path_2002 = datapath + '2002_annual_by_industry/2002.annual.by_industry/2002.annual 492 Couriers and messengers.csv'
path_2003 = datapath + '2003_annual_by_industry/2003.annual.by_industry/2003.annual 492 Couriers and messengers.csv'
path_2004 = datapath + '2004_annual_by_industry/2004.annual.by_industry/2004.annual 492 Couriers and messengers.csv'
path_2005 = datapath + '2005_annual_by_industry/2005.annual.by_industry/2005.annual 492 Couriers and messengers.csv'
path_2006 = datapath + '2006_annual_by_industry/2006.annual.by_industry/2006.annual 492 Couriers and messengers.csv'
path_2007 = datapath + '2007_annual_by_industry/2007.annual.by_industry/2007.annual 492 Couriers and messengers.csv'
path_2008 = datapath + '2008_annual_by_industry/2008.annual.by_industry/2008.annual 492 Couriers and messengers.csv'
path_2009 = datapath + '2009_annual_by_industry/2009.annual.by_industry/2009.annual 492 Couriers and messengers.csv'
path_2010 = datapath + '2010_annual_by_industry/2010.annual.by_industry/2010.annual 492 Couriers and messengers.csv'
path_2011 = datapath + '2011_annual_by_industry/2011.annual.by_industry/2011.annual 492 Couriers and messengers.csv'
path_2012 = datapath + '2012_annual_by_industry/2012.annual.by_industry/2012.annual 492 Couriers and messengers.csv'
path_2013 = datapath + '2013_annual_by_industry/2013.annual.by_industry/2013.annual 492 Couriers and messengers.csv'

In [6]:
df_2000 = pd.read_csv(path_2000)
df_2001 = pd.read_csv(path_2001)
df_2002 = pd.read_csv(path_2002)
df_2003 = pd.read_csv(path_2003)
df_2004 = pd.read_csv(path_2004)
df_2005 = pd.read_csv(path_2005)
df_2006 = pd.read_csv(path_2006)
df_2007 = pd.read_csv(path_2007)
df_2008 = pd.read_csv(path_2008)
df_2009 = pd.read_csv(path_2009)
df_2010 = pd.read_csv(path_2010)
df_2011 = pd.read_csv(path_2011)
df_2012 = pd.read_csv(path_2012)
df_2013 = pd.read_csv(path_2013)

In [7]:
#we want to keep the area_fips, industry_code, 'year', and annual_avg_emplvl columns 
df_2011.columns

Index(['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code',
       'year', 'qtr', 'disclosure_code', 'area_title', 'own_title',
       'industry_title', 'agglvl_title', 'size_title',
       'annual_avg_estabs_count', 'annual_avg_emplvl', 'total_annual_wages',
       'taxable_annual_wages', 'annual_contributions', 'annual_avg_wkly_wage',
       'avg_annual_pay', 'lq_disclosure_code', 'lq_annual_avg_estabs_count',
       'lq_annual_avg_emplvl', 'lq_total_annual_wages',
       'lq_taxable_annual_wages', 'lq_annual_contributions',
       'lq_annual_avg_wkly_wage', 'lq_avg_annual_pay', 'oty_disclosure_code',
       'oty_annual_avg_estabs_count_chg',
       'oty_annual_avg_estabs_count_pct_chg', 'oty_annual_avg_emplvl_chg',
       'oty_annual_avg_emplvl_pct_chg', 'oty_total_annual_wages_chg',
       'oty_total_annual_wages_pct_chg', 'oty_taxable_annual_wages_chg',
       'oty_taxable_annual_wages_pct_chg', 'oty_annual_contributions_chg',
       'oty_annual_contributions_pct_

In [8]:
#we want to keep the area_fips, industry_code, 'year', and 'annual_avg_emplvl' columns 
df_2016.columns

Index(['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code',
       'year', 'qtr', 'disclosure_code', 'annual_avg_estabs',
       'annual_avg_emplvl', 'total_annual_wages', 'taxable_annual_wages',
       'annual_contributions', 'annual_avg_wkly_wage', 'avg_annual_pay',
       'lq_disclosure_code', 'lq_annual_avg_estabs', 'lq_annual_avg_emplvl',
       'lq_total_annual_wages', 'lq_taxable_annual_wages',
       'lq_annual_contributions', 'lq_annual_avg_wkly_wage',
       'lq_avg_annual_pay', 'oty_disclosure_code', 'oty_annual_avg_estabs_chg',
       'oty_annual_avg_estabs_pct_chg', 'oty_annual_avg_emplvl_chg',
       'oty_annual_avg_emplvl_pct_chg', 'oty_total_annual_wages_chg',
       'oty_total_annual_wages_pct_chg', 'oty_taxable_annual_wages_chg',
       'oty_taxable_annual_wages_pct_chg', 'oty_annual_contributions_chg',
       'oty_annual_contributions_pct_chg', 'oty_annual_avg_wkly_wage_chg',
       'oty_annual_avg_wkly_wage_pct_chg', 'oty_avg_annual_pay_chg',
      

In [9]:
pdList = [df_2000, df_2001, df_2002, df_2003, df_2004, df_2005, df_2006, df_2007, df_2008, df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020]  # List of dataframes
df_merged = pd.concat(pdList)

In [10]:
df_merged = df_merged[['area_fips', 'industry_code', 'own_code', 'year', 'disclosure_code', 'annual_avg_emplvl', 'total_annual_wages']]

In [11]:
df_courier_emply = df_merged.rename(columns = {'annual_avg_emplvl': 'annual_avg_courier_emply'})

In [12]:
df_courier_emply

,area_fips,industry_code,own_code,year,disclosure_code,annual_avg_courier_emply,total_annual_wages
0,01000,492,5,2000,NaN,5900,168676535
1,01069,492,5,2000,NaN,158,4669079
2,01073,492,5,2000,NaN,1863,59033708
3,01089,492,5,2000,NaN,650,16778018
4,01097,492,5,2000,NaN,585,16360656
...,...,...,...,...,...,...,...
2179,C4966,492,5,2020,N,0,0
2180,C4970,492,5,2020,N,0,0
2181,C4974,492,5,2020,NaN,281,11363762
2182,US000,492,3,2020,N,0,0


Filters down to only private employment (own_code ==5)

In [13]:
df_courier_emply = df_courier_emply[df_courier_emply['own_code'] == 5]

This df has the annual average employment in the 492 NAICS code by county for years 2000 - 2020

In [14]:
df_courier_emply

,area_fips,industry_code,own_code,year,disclosure_code,annual_avg_courier_emply,total_annual_wages
0,01000,492,5,2000,NaN,5900,168676535
1,01069,492,5,2000,NaN,158,4669079
2,01073,492,5,2000,NaN,1863,59033708
3,01089,492,5,2000,NaN,650,16778018
4,01097,492,5,2000,NaN,585,16360656
...,...,...,...,...,...,...,...
2178,C4962,492,5,2020,NaN,1467,54568278
2179,C4966,492,5,2020,N,0,0
2180,C4970,492,5,2020,N,0,0
2181,C4974,492,5,2020,NaN,281,11363762


In [19]:
df_courier_emply.to_csv('QCEW_naics_492.csv')

No cases where data was suppressed and employment or wages were still published (as expected)

In [16]:
len(df_courier_emply[ (df_courier_emply['disclosure_code'] == 'N' ) & (df_courier_emply['annual_avg_courier_emply'] > 0)])

0

In [22]:
len(df_courier_emply[ (df_courier_emply['disclosure_code'] == 'N' ) & (df_courier_emply['total_annual_wages'] > 0)])

0

23k out of 43k (about half) cells were suppressed for this QCEW data, making annual_avg_courier_emply and total_annual_wages  show up as a zero

In [21]:
len(df_courier_emply)

43575

In [20]:
df_courier_emply[df_courier_emply['disclosure_code'] == 'N']

,area_fips,industry_code,own_code,year,disclosure_code,annual_avg_courier_emply,total_annual_wages
1,01001,492,5,2001,N,0,0
2,01003,492,5,2001,N,0,0
3,01009,492,5,2001,N,0,0
5,01025,492,5,2001,N,0,0
6,01027,492,5,2001,N,0,0
...,...,...,...,...,...,...,...
2173,C4890,492,5,2020,N,0,0
2174,C4902,492,5,2020,N,0,0
2176,C4934,492,5,2020,N,0,0
2179,C4966,492,5,2020,N,0,0


In [18]:
#making sure the industry code is the same for all entires - it is 
df_courier_emply.industry_code.unique()

array([492], dtype=int64)

In [36]:
df_courier_emply.dtypes

area_fips                   object
industry_code                int64
own_code                     int64
year                         int64
annual_avg_courier_emply     int64
total_annual_wages           int64
dtype: object

In [37]:
df_courier_emply.isna().sum()

area_fips                   0
industry_code               0
own_code                    0
year                        0
annual_avg_courier_emply    0
total_annual_wages          0
dtype: int64

In [23]:
df_courier_emply[df_courier_emply['annual_avg_courier_emply'] > 0].groupby('year').size()

year
2000     691
2001     826
2002     873
2003     904
2004     889
2005     917
2006     918
2007     902
2008     950
2009     942
2010     956
2011    1019
2012    1005
2013    1016
2014    1032
2015    1021
2016     999
2017    1016
2018     991
2019     982
2020    1004
dtype: int64

In [24]:
df_courier_emply[(df_courier_emply['annual_avg_courier_emply'] == 0) & (df_courier_emply['year'] == 2020)].groupby('area_fips').size()

area_fips
01001    1
01013    1
01025    1
01027    1
01031    1
        ..
C4890    1
C4902    1
C4934    1
C4966    1
C4970    1
Length: 1171, dtype: int64

Next step:
make sure industry code is the same for all years
check for NAs
do the same for couriers 
need to account for state, local, and private employment
do due dilligence on the change in method for getting the data from 2013 to 2014
Merge this onto the main df, but only want to keep rows (counties) that exists in the main df

In [25]:
df_courier_emply[df_courier_emply['area_fips'] == '01000']

,area_fips,industry_code,own_code,year,annual_avg_courier_emply
0,01000,492,5,2000,5900
0,01000,492,5,2001,5762
0,01000,492,5,2002,5538
0,01000,492,5,2003,5449
0,01000,492,5,2004,5315
0,01000,492,5,2005,5455
0,01000,492,5,2006,5581
0,01000,492,5,2007,5604
0,01000,492,5,2008,5650
0,01000,492,5,2009,5160


In [26]:
df_courier_emply[df_courier_emply['area_fips'] == '25009']

,area_fips,industry_code,own_code,year,annual_avg_courier_emply
200,25009,492,5,2000,619
642,25009,492,5,2001,600
653,25009,492,5,2002,583
665,25009,492,5,2003,569
670,25009,492,5,2004,595
667,25009,492,5,2005,538
673,25009,492,5,2006,540
664,25009,492,5,2007,511
683,25009,492,5,2008,519
669,25009,492,5,2009,493


In [27]:
df_courier_emply[df_courier_emply['area_fips'] == '12031']

,area_fips,industry_code,own_code,year,annual_avg_courier_emply
80,12031,492,5,2000,4522
207,12031,492,5,2001,4690
212,12031,492,5,2002,4554
215,12031,492,5,2003,4704
221,12031,492,5,2004,4727
216,12031,492,5,2005,5088
218,12031,492,5,2006,4890
217,12031,492,5,2007,4927
219,12031,492,5,2008,4880
218,12031,492,5,2009,4569


In [28]:
df_courier_emply[df_courier_emply['area_fips'] == '08001']

,area_fips,industry_code,own_code,year,annual_avg_courier_emply
53,08001,492,5,2000,4380
144,08001,492,5,2001,4446
147,08001,492,5,2002,4185
149,08001,492,5,2003,3917
155,08001,492,5,2004,3862
151,08001,492,5,2005,3913
152,08001,492,5,2006,4029
149,08001,492,5,2007,4055
153,08001,492,5,2008,3935
151,08001,492,5,2009,3718
